In [ ]:
import os
import torch
from torchvision.datasets import WIDERFace
from torchvision.transforms import ToTensor
from PIL import Image

# Instalar ultralytics se não tiver
# !pip install ultralytics

from ultralytics import YOLO

# =============================
# 1. Configuração
# =============================
base_dir = "widerface_yolo"
os.makedirs(base_dir, exist_ok=True)

splits = {"train": 500, "val": 200}  # quantidade de imagens

# =============================
# 2. Função para converter para YOLO
# =============================
def convert_to_yolo(dataset, split, max_images=500):
    out_img_dir = os.path.join(base_dir, "images", split)
    out_lbl_dir = os.path.join(base_dir, "labels", split)
    os.makedirs(out_img_dir, exist_ok=True)
    os.makedirs(out_lbl_dir, exist_ok=True)

    for idx in range(min(len(dataset), max_images)):
        img, target = dataset[idx]
        w, h = img.size

        # Salvar imagem
        img_path = os.path.join(out_img_dir, f"{idx}.jpg")
        img.save(img_path)

        # Criar arquivo de label no formato YOLO
        label_path = os.path.join(out_lbl_dir, f"{idx}.txt")
        with open(label_path, "w") as f:
            for box in target["bbox"]:
                x, y, bw, bh = box.tolist()
                # Normalizar para YOLO: class cx cy w h
                cx = (x + bw / 2) / w
                cy = (y + bh / 2) / h
                nw = bw / w
                nh = bh / h
                f.write(f"0 {cx} {cy} {nw} {nh}\n")

        if idx % 100 == 0:
            print(f"[{split}] Convertidas {idx}/{max_images} imagens")

# =============================
# 3. Baixar e converter dataset
# =============================
for split, n_imgs in splits.items():
    dataset = WIDERFace(root=".", split=split, transform=None, download=True)
    convert_to_yolo(dataset, split, n_imgs)

# =============================
# 4. Criar arquivo dataset.yaml
# =============================
yaml_path = os.path.join(base_dir, "widerface.yaml")
with open(yaml_path, "w") as f:
    f.write(f"""
path: {os.path.abspath(base_dir)}
train: images/train
val: images/val

names:
  0: face
""")

print(f"Arquivo de configuração criado em {yaml_path}")

# =============================
# 5. Treinar YOLOv8
# =============================
model = YOLO("yolov8n.pt")  # modelo pequeno p/ teste rápido
model.train(data=yaml_path, epochs=5, imgsz=416, batch=8)

# =============================
# 6. Testar em uma imagem do validation
# =============================
test_img = os.path.join(base_dir, "images/val/0.jpg")
results = model.predict(test_img, save=True, conf=0.25)

print("Resultado salvo em runs/detect/predict/")
